# Notebook for Programming in Problem 2


## Learning Objectives
In this problem, we will use [PyTorch](https://pytorch.org/) to implement long short-term memory (LSTM) for named entity recognition (NER). We will use the provided dataset 'eng.train' and 'eng.val'.

## Writing Code
Look for the keyword "TODO" and fill in your code in the empty space.
Feel free to change function signatures, but be careful that you might need to also change how they are called in other parts of the notebook.

In [ ]:
!nvidia-smi # you may need to try reconnecting to get a T4 gpu

Sat Apr 27 13:47:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving eng.train to eng.train


## Installing PyTorch and Other Packages

Install PyTorch using pip. See [https://pytorch.org/](https://pytorch.org/) if you want to install it on your computer.

In [ ]:
!pip install torch torchtext -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64

Test if our installation works:

In [ ]:
import torch

# Multiply two matrices on GPU
a = torch.rand(100, 200).cuda()
b = torch.rand(200, 100).cuda()
c = torch.matmul(a, b)

print("PyTorch successfully installed!")
print("Version:", torch.__version__)

PyTorch successfully installed!
Version: 2.2.1+cu121


Also install [scikit-learn](https://scikit-learn.org/stable/). We will use it for calculating evaluation metrics such as accuracy and F1 score.

In [ ]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 37.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


Let's import all the packages at once:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import Vocab, vocab
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import re
from collections import Counter
from typing import List, Tuple, Dict, Optional, Any

## Long Short Term Memory (LSTM)

### Data Loading

We will use the provided datasets for named entity recognition ('eng.train' & 'eng.val'). Take a look at the first 50 lines:

In [ ]:
!cat eng.train | head -n 50

EU NNP I-NP ORG
rejects VBZ I-VP O
German JJ I-NP MISC
call NN I-NP O
to TO I-VP O
boycott VB I-VP O
British JJ I-NP MISC
lamb NN I-NP O
. . O O

Peter NNP I-NP PER
Blackburn NNP I-NP PER

BRUSSELS NNP I-NP LOC
1996-08-22 CD I-NP O

The DT I-NP O
European NNP I-NP ORG
Commission NNP I-NP ORG
said VBD I-VP O
on IN I-PP O
Thursday NNP I-NP O
it PRP B-NP O
disagreed VBD I-VP O
with IN I-PP O
German JJ I-NP MISC
advice NN I-NP O
to TO I-PP O
consumers NNS I-NP O
to TO I-VP O
shun VB I-VP O
British JJ I-NP MISC
lamb NN I-NP O
until IN I-SBAR O
scientists NNS I-NP O
determine VBP I-VP O
whether IN I-SBAR O
mad JJ I-NP O
cow NN I-NP O
disease NN I-NP O
can MD I-VP O
be VB I-VP O
transmitted VBN I-VP O
to TO I-PP O
sheep NN I-NP O
. . O O

Germany NNP I-NP LOC
's POS B-NP O
representative NN I-NP O


Each line corresponds to a word. Different sentences are separated by an additional line break. Take "EU NNP I-NP ORG" as an example. "EU" is a word. "NNP" and "I-NP" are tags for POS tagging and chunking, which we will ignore. "ORG" is the tag for NER, which is our prediction target. There are 5 possible values for the NER tag: ORG, PER, LOC, MISC, and O.



In [ ]:
# A sentence is a list of (word, tag) tuples.
# For example, [("hello", "O"), ("world", "O"), ("!", "O")]
Sentence = List[Tuple[str, str]]


def read_data_file(
    datapath: str,
) -> Tuple[List[Sentence], Dict[str, int], Dict[str, int]]:
    """
    Read and preprocess input data from the file `datapath`.
    Example:
    ```
        sentences, word_cnt, tag_cnt = read_data_file("eng.train")
    ```
    Return values:
        `sentences`: a list of sentences, including words and NER tags
        `word_cnt`: a Counter object, the number of occurrences of each word
        `tag_cnt`: a Counter object, the number of occurences of each NER tag
    """
    sentences: List[Sentence] = []
    word_cnt: Dict[str, int] = Counter()
    tag_cnt: Dict[str, int] = Counter()

    for sentence_txt in open(datapath).read().split("\n\n"):
        if "DOCSTART" in sentence_txt:
            # Ignore dummy sentences at the begining of each document.
            continue
        # Read a new sentence
        sentences.append([])
        for token in sentence_txt.split("\n"):
            w, _, _, t = token.split()
            # Replace all digits with "0" to reduce out-of-vocabulary words
            w = re.sub("\d", "0", w)
            word_cnt[w] += 1
            tag_cnt[t] += 1
            sentences[-1].append((w, t))

    return sentences, word_cnt, tag_cnt



In [ ]:
# Some helper code
def get_device() -> torch.device:
    """
    Use GPU when it is available; use CPU otherwise.
    See https://pytorch.org/docs/stable/notes/cuda.html#device-agnostic-code
    """
    return torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [ ]:
def eval_metrics(ground_truth: List[int], predictions: List[int]) -> Dict[str, Any]:
    """
    Calculate various evaluation metrics such as accuracy and F1 score
    Parameters:
        `ground_truth`: the list of ground truth NER tags
        `predictions`: the list of predicted NER tags
    """
    f1_scores = f1_score(ground_truth, predictions, average=None)
    return {
        "accuracy": accuracy_score(ground_truth, predictions),
        "f1": f1_scores,
        "average f1": np.mean(f1_scores),
        "confusion matrix": confusion_matrix(ground_truth, predictions),
    }

## Long Short-term Memory (LSTM)

Now we implement an one-layer LSTM for the NER task.

### Data Loading **(4 points)**

We first implement the data loader. In the provided datasets, each data example is a variable-length sentence. How can we pack multiple sentences with different lengths into the same batch? One possible solution is to pad them to the same length using a special token. The code below illustrates the idea:

In [ ]:
# 3 sentences with different lengths
sentence_1 = torch.tensor([6, 1, 2])
sentence_2 = torch.tensor([4, 2, 7, 7, 9])
sentence_3 = torch.tensor([3, 4])
# Form a batch by padding 0
sentence_batch = torch.tensor([
    [6, 1, 2, 0, 0],
    [4, 2, 7, 7, 9],
    [3, 4, 0, 0, 0],
])

We implement the above idea in a customized batching function `form_batch`. Optionally, see [here](https://pytorch.org/docs/stable/data.html#loading-batched-and-non-batched-data) for how batching works in PyTorch.

In [ ]:
class SequenceDataset(Dataset):
    """
    Each data example is a sentence, including its words and NER tags.
    """

    def __init__(
        self, datapath: str, words_vocab: Optional[Vocab] = None, tags_vocab: Optional[Vocab] = None
    ) -> None:
        """
        Initialize the dataset by reading from datapath.
        """
        super().__init__()
        self.sentences: List[Sentence] = []
        UNKNOWN = "<UNKNOWN>"
        PAD = "<PAD>"  # Special token used for padding

        print("Loading data from %s" % datapath)
        self.sentences, word_cnt, tag_cnt = read_data_file(datapath)
        print("%d sentences loaded." % len(self.sentences))

        if words_vocab is None:
            words_vocab = vocab(word_cnt, specials=[PAD, UNKNOWN])
            words_vocab.set_default_index(words_vocab[UNKNOWN])

        self.words_vocab = words_vocab

        self.unknown_idx = self.words_vocab[UNKNOWN]
        self.pad_idx = self.words_vocab[PAD]

        if tags_vocab is None:
            tags_vocab = vocab(tag_cnt, specials=[])
        self.tags_vocab = tags_vocab

    def __getitem__(self, idx: int) -> Sentence:
        """
        Get the idx'th sentence in the dataset.
        """
        return self.sentences[idx]

    def __len__(self) -> int:
        """
        Return the number of sentences in the dataset.
        """
        # TODO: Implement this method
        # START HERE
        return len(self.sentences)
        #raise NotImplementedError
        # END

    def form_batch(self, sentences: List[Sentence]) -> Dict[str, Any]:
        """
        A customized function for batching a number of sentences together.
        Different sentences have different lengths. Let max_len be the longest length.
        When packing them into one tensor, we need to pad all sentences to max_len.
        Return values:
            `words`: a list in which each element itself is a list of words in a sentence
            `word_idxs`: a batch_size x max_len tensor.
                       word_idxs[i][j] is the index of the j'th word in the i'th sentence .
            `tags`: a list in which each element itself is a list of tags in a sentence
            `tag_idxs`: a batch_size x max_len tensor
                      tag_idxs[i][j] is the index of the j'th tag in the i'th sentence.
            `valid_mask`: a batch_size x max_len tensor
                        valid_mask[i][j] is True if the i'th sentence has the j'th word.
                        Otherwise, valid[i][j] is False.
        """
        words: List[List[str]] = []
        tags: List[List[str]] = []
        max_len = -1  # length of the longest sentence
        for sent in sentences:
            words.append([])
            tags.append([])
            for w, t in sent:
                words[-1].append(w)
                tags[-1].append(t)
            max_len = max(max_len, len(words[-1]))

        batch_size = len(sentences)
        word_idxs = torch.full(
            (batch_size, max_len), fill_value=self.pad_idx, dtype=torch.int64
        )
        tag_idxs = torch.full_like(word_idxs, fill_value=self.tags_vocab["O"])
        valid_mask = torch.zeros_like(word_idxs, dtype=torch.bool)

        ## TODO: Fill in the values in word_idxs, tag_idxs, and valid_mask
        ## Caveat: There may be out-of-vocabulary words in validation data
        ## See torchtext.vocab.Vocab: https://pytorch.org/text/stable/vocab.html#torchtext.vocab.Vocab
        ## START HERE
        for i, sent in enumerate(sentences):
            for j, (word, tag) in enumerate(sent):
                if word in self.words_vocab:
                    word_index = self.words_vocab[word]
                else:
                    word_index = self.unknown_idx

        tag_index = self.tags_vocab[tag]  # 태그 인덱스는 이미 존재한다고 가정

        word_idxs[i, j] = word_index
        tag_idxs[i, j] = tag_index
        valid_mask[i, j] = True


        #raise NotImplementedError

        # END

        return {
            "words": words,
            "word_idxs": word_idxs,
            "tags": tags,
            "tag_idxs": tag_idxs,
            "valid_mask": valid_mask,
        }


def create_sequence_dataloaders(
    batch_size: int, shuffle: bool = True
) -> Tuple[DataLoader, DataLoader, Vocab]:
    """
    Create the dataloaders for training and validaiton.
    """
    ds_train = SequenceDataset("eng.train")
    ds_val = SequenceDataset("eng.val", words_vocab=ds_train.words_vocab, tags_vocab=ds_train.tags_vocab)
    loader_train = DataLoader(
        ds_train,
        batch_size,
        shuffle,
        collate_fn=ds_train.form_batch,  # customized function for batching
        drop_last=True,
        pin_memory=True,
    )
    loader_val = DataLoader(
        ds_val, batch_size, collate_fn=ds_val.form_batch, pin_memory=True
    )
    return loader_train, loader_val, ds_train

Here is a simple sanity-check. Try to understand its output.

In [ ]:
def check_sequence_dataloader() -> None:
    loader_train, _, _ = create_sequence_dataloaders(batch_size=3, shuffle=False)
    print("Iterating on the training data..")
    for i, data_batch in enumerate(loader_train):
        if i == 0:
            print(data_batch)
    print("Done!")


check_sequence_dataloader()

Loading data from eng.train
14041 sentences loaded.
Loading data from eng.val
3490 sentences loaded.
Iterating on the training data..
{'words': [['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], ['Peter', 'Blackburn'], ['BRUSSELS', '0000-00-00']], 'word_idxs': tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 14,  0,  0,  0,  0,  0,  0,  0]]), 'tags': [['ORG', 'O', 'MISC', 'O', 'O', 'O', 'MISC', 'O', 'O'], ['PER', 'PER'], ['LOC', 'O']], 'tag_idxs': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'valid_mask': tensor([[False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False],
        [False,  True, False, False, False, False, False, False, False]])}
Done!


### Implement the Model **(8 points)**

Next, implement LSTM for predicting NER tags from input words. [nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) is definitely useful. Further, it is tricky to handle sentences in the same batch with different lengths. Please read the PyTorch documentation in detail!


In [ ]:
class LSTM(nn.Module):
    """
    Long short-term memory for NER
    """

    def __init__(self, words_vocab: Vocab, tags_vocab:Vocab, d_emb: int, d_hidden: int, bidirectional: bool) -> None:
        """
        Initialize an LSTM
        Parameters:
            `words_vocab`: vocabulary of words
            `tags_vocab`: vocabulary of tags
            `d_emb`: dimension of word embeddings (D)
            `d_hidden`: dimension of the hidden layer (H)
            `bidirectional`: true if LSTM should be bidirectional
        """
        super().__init__()
        # TODO: Create the word embeddings (nn.Embedding),
        #       the LSTM (nn.LSTM) and the output layer (nn.Linear).
        #       Read the torch docs for additional guidance : https://pytorch.org/docs/stable
        #       Note: Pay attention to the LSTM output shapes!
        # START HERE
        self.embedding = nn.Embedding(num_embeddings=len(words_vocab), embedding_dim=d_emb)
        self.lstm = nn.LSTM(input_size=d_emb, hidden_size=d_hidden, bidirectional=bidirectional, batch_first=True)
        multiplier = 2 if bidirectional else 1
        self.output_layer = nn.Linear(d_hidden * multiplier, len(tags_vocab))


        #raise NotImplementedError

        # END

    def forward(
        self, word_idxs: torch.Tensor, valid_mask: torch.Tensor
    ) -> torch.Tensor:
        """
        Given words in sentences, predict the logits of the NER tag.
        Parameters:
            `word_idxs`: a batch_size x max_len tensor
            `valid_mask`: a batch_size x max_len tensor
        Return values:
            `logits`: a batch_size x max_len x 5 tensor
        """
        # TODO: Implement the forward pass
        # START HERE
        embeddings = self.embedding(word_idxs)
        lstm_output, _ = self.lstm(embeddings)
        logits = self.output_layer(lstm_output)

        #raise NotImplementedError

        # END
        return logits

We do a sanity-check by loading a batch of data examples and pass it through the network.

In [ ]:
def check_lstm() -> None:
    # Hyperparameters
    batch_size = 4
    d_emb = 64
    d_hidden = 128
    bidirectional = True
    # Create the dataloaders and the model
    loader_train, _, ds_train = create_sequence_dataloaders(batch_size)
    model = LSTM(ds_train.words_vocab, ds_train.tags_vocab, d_emb, d_hidden, bidirectional)
    device = get_device()
    model.to(device)
    print(model)
    # Get the first batch
    data_batch = next(iter(loader_train))
    # Move data to GPU
    word_idxs = data_batch["word_idxs"].to(device, non_blocking=True)
    tag_idxs = data_batch["tag_idxs"].to(device, non_blocking=True)
    valid_mask = data_batch["valid_mask"].to(device, non_blocking=True)
    # Calculate the model
    print("Input word_idxs shape:", word_idxs.size())
    print("Input valid_mask shape:", valid_mask.size())
    logits = model(word_idxs, valid_mask)
    print("Output logits shape:", logits.size())


check_lstm()

Loading data from eng.train
14041 sentences loaded.
Loading data from eng.val
3490 sentences loaded.
LSTM(
  (embedding): Embedding(20102, 64)
  (lstm): LSTM(64, 128, batch_first=True, bidirectional=True)
  (output_layer): Linear(in_features=256, out_features=5, bias=True)
)
Input word_idxs shape: torch.Size([4, 20])
Input valid_mask shape: torch.Size([4, 20])
Output logits shape: torch.Size([4, 20, 5])


### Training and Validation **(6 points)**

Complete the functions for training and validating the LSTM model. When calculating the loss function, you only want to include values from valid positions (where `valid_mask` is `True`). The `reduction` parameter in [F.cross_entropy](https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.cross_entropy) may be useful.

In [ ]:
def train_lstm(
    model: nn.Module,
    loader: DataLoader,
    optimizer: optim.Optimizer,
    device: torch.device,
    silent: bool = False,  # whether to print the training loss
) -> Tuple[float, Dict[str, Any]]:
    """
    Train the LSTM model.
    Return values:
        1. the average training loss
        2. training metrics such as accuracy and F1 score
    """
    model.train()
    ground_truth = []
    predictions = []
    losses = []
    report_interval = 100

    for i, data_batch in enumerate(loader):
        word_idxs = data_batch["word_idxs"].to(device, non_blocking=True)
        tag_idxs = data_batch["tag_idxs"].to(device, non_blocking=True)
        valid_mask = data_batch["valid_mask"].to(device, non_blocking=True)

        # TODO:
        # 1. Perform the forward pass to calculate the model's output. Save it to the variable "logits".
        # 2. Calculate the loss using the output and the ground truth tags. Save it to the variable "loss".
        # 3. Perform the backward pass to calculate the gradient.
        # 4. Use the optimizer to update model parameters.
        # Caveat:
        # 1. You may need to call optimizer.zero_grad(). Figure out what it does!
        # 2. When calculating the loss, you should only consider positions where valid_mask == True
        # START HERE

        # logits = ...
        # loss = ...
        optimizer.zero_grad()  # 루프 시작에서 그라디언트를 초기화

        logits = model(word_idxs, valid_mask)  # 모델의 forward pass 실행
        loss_fn = nn.CrossEntropyLoss()  # Loss function 정의

    # logits에서 valid_mask가 True인 위치만 손실을 계산합니다.
        active_logits = logits.view(-1, logits.shape[-1])[valid_mask.view(-1)]
        active_labels = tag_idxs.view(-1)[valid_mask.view(-1)]
        loss = loss_fn(active_logits, active_labels)

        loss.backward()  # 그라디언트 계산
        optimizer.step()  # 파라미터 업데이트

        losses.append(loss.item())
        #raise NotImplementedError
        # END

       # optimizer.zero_grad()
       # loss.backward()
       # optimizer.step()

       # losses.append(loss.item())

        # we get (unmasked) predictions by getting argmax of logits along last dimension (You will need to define logits!)
        net_predictions = torch.argmax(logits, -1)

        # flattening a tensor simply converts it from a multi-dimensional to a single-dimensional tensor; we flatten here to make it easier to extract ground truths and predictions
        tag_idxs_flat = tag_idxs.flatten()
        valid_mask_flat = valid_mask.flatten()
        net_predictions_flat = net_predictions.flatten()

        ground_truth.extend(tag_idxs_flat[valid_mask_flat].tolist())
        predictions.extend(net_predictions_flat[valid_mask_flat].tolist())

        if not silent and i > 0 and i % report_interval == 0:
            print(
                "\t[%06d/%06d] Loss: %f"
                % (i, len(loader), np.mean(losses[-report_interval:]))
            )

    return np.mean(losses), eval_metrics(ground_truth, predictions)


def validate_lstm(
    model: nn.Module, loader: DataLoader, device: torch.device
) -> Tuple[float, Dict[str, Any]]:
    """
    Validate the model.
    Return the validation loss and metrics.
    """
    model.eval()
    ground_truth = []
    predictions = []
    losses = []

    with torch.no_grad():

        for data_batch in loader:
            word_idxs = data_batch["word_idxs"].to(device, non_blocking=True)
            tag_idxs = data_batch["tag_idxs"].to(device, non_blocking=True)
            valid_mask = data_batch["valid_mask"].to(device, non_blocking=True)

            # TODO: Similar to what you did in train_lstm, but only step 1 and 2.
            # Caveat: When calculating the loss, you should only consider positions where valid_mask == True

            # START HERE
            logits = model(word_idxs, valid_mask)
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits[valid_mask], tag_idxs[valid_mask])

            #raise NotImplementedError
            # END

            losses.append(loss.item())

            # we get (unmasked) predictions by getting argmax of logits (You will need to define logits!)
            net_predictions = torch.argmax(logits, -1)

            # flattening a tensor simply converts it from a multi-dimensional to a single-dimensional tensor; we flatten here to make it easier to extract ground truths and predictions
            tag_idxs_flat = tag_idxs.flatten()
            valid_mask_flat = valid_mask.flatten()
            net_predictions_flat = net_predictions.flatten()

            ground_truth.extend(tag_idxs_flat[valid_mask_flat].tolist())
            predictions.extend(net_predictions_flat[valid_mask_flat].tolist())

    return np.mean(losses), eval_metrics(ground_truth, predictions)


def train_val_loop_lstm(hyperparams: Dict[str, Any]) -> None:
    """
    Train and validate the LSTM model for a number of epochs.
    """
    print("Hyperparameters:", hyperparams)
    # Create the dataloaders
    loader_train, loader_val, ds_train = create_sequence_dataloaders(
        hyperparams["batch_size"]
    )
    # Create the model
    model = LSTM(
        ds_train.words_vocab,
        ds_train.tags_vocab,
        hyperparams["d_emb"],
        hyperparams["d_hidden"],
        hyperparams["bidirectional"],
    )
    device = get_device()
    model.to(device)
    print(model)
    # Create the optimizer
    optimizer = optim.RMSprop(
        model.parameters(), hyperparams["learning_rate"], weight_decay=hyperparams["l2"]
    )

    # Train and validate
    for i in range(hyperparams["num_epochs"]):
        print("Epoch #%d" % i)

        print("Training..")
        loss_train, metrics_train = train_lstm(model, loader_train, optimizer, device)
        print("Training loss: ", loss_train)
        print("Training metrics:")
        for k, v in metrics_train.items():
            print("\t", k, ": ", v)

        print("Validating..")
        loss_val, metrics_val = validate_lstm(model, loader_val, device)
        print("Validation loss: ", loss_val)
        print("Validation metrics:")
        for k, v in metrics_val.items():
            print("\t", k, ": ", v)

    print("Done!")

Run the experiment:

In [ ]:
train_val_loop_lstm({
    "bidirectional": True,
    "batch_size": 512,
    "d_emb": 64,
    "d_hidden": 128,
    "num_epochs": 15,
    "learning_rate": 0.005,
    "l2": 1e-6,
})

Hyperparameters: {'bidirectional': True, 'batch_size': 512, 'd_emb': 64, 'd_hidden': 128, 'num_epochs': 15, 'learning_rate': 0.005, 'l2': 1e-06}
Loading data from eng.train
14041 sentences loaded.
Loading data from eng.val
3490 sentences loaded.
LSTM(
  (embedding): Embedding(20102, 64)
  (lstm): LSTM(64, 128, batch_first=True, bidirectional=True)
  (output_layer): Linear(in_features=256, out_features=5, bias=True)
)
Epoch #0
Training..
Training loss:  0.7123185556773495
Training metrics:
	 accuracy :  0.8888888888888888
	 f1 :  [0.         0.94117647]
	 average f1 :  0.47058823529411764
	 confusion matrix :  [[ 0  1]
 [ 2 24]]
Validating..


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Validation loss:  0.002816062154514449
Validation metrics:
	 accuracy :  1.0
	 f1 :  [1.]
	 average f1 :  1.0
	 confusion matrix :  [[7]]
Epoch #1
Training..
Training loss:  0.48893464070365383
Training metrics:
	 accuracy :  0.9259259259259259
	 f1 :  [0.         0.96153846 0.        ]
	 average f1 :  0.32051282051282054
	 confusion matrix :  [[ 0  1  0]
 [ 0 25  0]
 [ 0  1  0]]
Validating..


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Validation loss:  0.04941164780341621
Validation metrics:
	 accuracy :  1.0
	 f1 :  [1.]
	 average f1 :  1.0
	 confusion matrix :  [[7]]
Epoch #2
Training..


...

Training loss:  0.24266288394574076
Training metrics:
	 accuracy :  0.9259259259259259
	 f1 :  [0.         0.96153846 0.        ]
	 average f1 :  0.32051282051282054
	 confusion matrix :  [[ 0  1  0]
 [ 0 25  0]
 [ 0  1  0]]
Validating..
Validation loss:  0.10206735215615481
Validation metrics:
	 accuracy :  1.0
	 f1 :  [1.]
	 average f1 :  1.0
	 confusion matrix :  [[7]]
Epoch #13
Training..


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Training loss:  0.048072458697586425
Training metrics:
	 accuracy :  1.0
	 f1 :  [1.]
	 average f1 :  1.0
	 confusion matrix :  [[27]]
Validating..
Validation loss:  0.03445148276348066
Validation metrics:
	 accuracy :  1.0
	 f1 :  [1.]
	 average f1 :  1.0
	 confusion matrix :  [[7]]
Epoch #14
Training..


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Training loss:  0.016939781794024857
Training metrics:
	 accuracy :  1.0
	 f1 :  [1.]
	 average f1 :  1.0
	 confusion matrix :  [[27]]
Validating..
Validation loss:  0.005680834618812826
Validation metrics:
	 accuracy :  1.0
	 f1 :  [1.]
	 average f1 :  1.0
	 confusion matrix :  [[7]]
Done!


We were using bidirectional LSTMs. Please re-run the experiment with a regular (unidirectional) LSTM.

In [ ]:
## TODO: Re-run with unidirectional LSTMs
## Keep other hyperparameters fixed
train_val_loop_lstm({
    "bidirectional": False,
    "batch_size": 512,
    "d_emb": 64,
    "d_hidden": 128,
    "num_epochs": 15,
    "learning_rate": 0.005,
    "l2": 1e-6,
})
## END

Hyperparameters: {'bidirectional': False, 'batch_size': 512, 'd_emb': 64, 'd_hidden': 128, 'num_epochs': 15, 'learning_rate': 0.005, 'l2': 1e-06}
Loading data from eng.train
14041 sentences loaded.
Loading data from eng.val
3490 sentences loaded.
LSTM(
  (embedding): Embedding(20102, 64)
  (lstm): LSTM(64, 128, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=5, bias=True)
)
Epoch #0
Training..
Training loss:  0.059087134210804595
Training metrics:
	 accuracy :  0.9629629629629629
	 f1 :  [0.98113208 0.        ]
	 average f1 :  0.49056603773584906
	 confusion matrix :  [[26  1]
 [ 0  0]]
Validating..


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Validation loss:  4.216505320593049e-05
Validation metrics:
	 accuracy :  1.0
	 f1 :  [1.]
	 average f1 :  1.0
	 confusion matrix :  [[7]]
Epoch #1
Training..


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Training loss:  3.2702539640552086e-05
Training metrics:
	 accuracy :  1.0
	 f1 :  [1.]
	 average f1 :  1.0
	 confusion matrix :  [[27]]
Validating..


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Training loss:  0.2440217368394197
Training metrics:
	 accuracy :  0.9259259259259259
	 f1 :  [0.96153846 0.         0.        ]
	 average f1 :  0.32051282051282054
	 confusion matrix :  [[25  0  0]
 [ 1  0  0]
 [ 1  0  0]]
Validating..
Validation loss:  0.20946933714107477
Validation metrics:
	 accuracy :  1.0
	 f1 :  [1.]
	 average f1 :  1.0
	 confusion matrix :  [[7]]
Epoch #14
Training..


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Training loss:  0.14465959793866473
Training metrics:
	 accuracy :  0.9629629629629629
	 f1 :  [0.98113208 0.        ]
	 average f1 :  0.49056603773584906
	 confusion matrix :  [[26  0]
 [ 1  0]]
Validating..
Validation loss:  0.13117362706023933
Validation metrics:
	 accuracy :  1.0
	 f1 :  [1.]
	 average f1 :  1.0
	 confusion matrix :  [[7]]
Done!


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


### Questions **(2 points)**

How does bidirectional LSTMs compare to unidirectional LSTMs? Why?

**TODO: Please fill in your answer here**

---answer----
bidirectional LSTMs은 한 시퀀스는 시작부터 끝, 다른 시퀀스는 끝부터 시작으로 작업을 합니다. 제일 큰 특징의 경우, 전 시점의 시퀀스에 대해서 앞뒤 정보를 모두 획득이 가능하다는 점입니다. 주로 전체맥락이 중요한경우에 많이 이용합니다.
장점은 데이터를 양방향으로 처리해서 단방향접근시 놓칠수 있는 패턴을 포착이 가능하단점이 있습니다. 이번 과제와 같이 전 시점의 패턴이 중요할경우, bidirectional LSTMs을 쓰는 것이 좋습니다. 단점으로는 계산적으로 단방향보다 비용과 시간이 많이드는(약 2배) 점입니다.

반면 unidirectional LSTMs은 처음부터 끝 까지, 즉 사람이 글을 읽어 내려가는 방식과 같이 정방향으로만 작업이 가능하며, 과거와 현재의 데이터만 사용해서 예측을 실행합니다.
한계는 과거의 정보만 이용할수 있어서정확한 예측을 위해서 미래의 입력특징의 맥락이 중요할 경우, 성능이 안 좋다.
주로 과거의 데이터를 주로 쓰는 시계열데이터에 많이 씁니다.

데이터의 전체적인 맥락이 중요할경우 bidirectional LSTMs을 쓰고, 실시간처리나 과거의 데이터가 중요한 경우 unidirectional LSTMs을 사용하는것이 효율적입니다.
코드로 구현시
uni -> nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=False)
bi -> nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True)
uni는 bidirectional=False, bi는 bidirectional=True가 됩니다.
hidden_dim의경우 단방향이 1일때, 양방향은 2가 되며 2배의 차이가 납니다.


In [ ]:

---answer---- bidirectional LSTMs은 한 시퀀스는 시작부터 끝, 다른 시퀀스는 끝부터 시작으로 작업을 합니다. 제일 큰 특징의 경우, 전 시점의 시퀀스에 대해서 앞뒤 정보를 모두 획득이 가능하다는 점입니다. 주로 전체맥락이 중요한경우에 많이 이용합니다.
장점은 데이터를 양방향으로 처리해서 단방향접근시 놓칠수 있는 패턴을 포착이 가능하단점이 있습니다. 이번 과제와 같이 전 시점의 패턴이 중요할경우, bidirectional LSTMs을 쓰는 것이 좋습니다. 단점으로는 계산적으로 단방향보다 비용과 시간이 많이드는(약 2배) 점입니다.

반면 unidirectional LSTMs은 처음부터 끝 까지, 즉 사람이 글을 읽어 내려가는 방식과 같이 정방향으로만 작업이 가능하며, 과거와 현재의 데이터만 사용해서 예측을 실행합니다.
한계는 과거의 정보만 이용할수 있어서정확한 예측을 위해서 미래의 입력특징의 맥락이 중요할 경우, 성능이 안 좋다.
주로 과거의 데이터를 주로 쓰는 시계열데이터에 많이 씁니다.

데이터의 전체적인 맥락이 중요할경우 bidirectional LSTMs을 쓰고, 실시간처리나 과거의 데이터가 중요한 경우 unidirectional LSTMs을 사용하는것이 효율적입니다.
코드로 구현시
uni -> nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=False)
bi -> nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True)
uni는 bidirectional=False, bi는 bidirectional=True가 됩니다.
hidden_dim의경우 단방향이 1일때, 양방향은 2가 되며 2배의 차이가 납니다.